In [25]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import itertools

# importing the Yelp data
def import_data():
    # Load a csv of floats:
    X = np.genfromtxt("yelp_data/upvote_data.csv", delimiter=",")
    # Load a text file of integers:
    y = np.loadtxt("yelp_data/upvote_labels.txt", dtype=np.int)
    # Load a text file of strings:
    featureNames = open("yelp_data/upvote_features.txt").read().splitlines()
    
    return X,y,featureNames

# computs the value of the objective function that is minimized
def fn_opt(X,w,y,b,lmd):
    val = np.matmul(X,w)+b-y
    return np.sum(val*val) + lmd * np.sum(np.absolute(w))

# Performs coordinated descent when the value of lambda is fixed
def lasso_fixlmd(X,w,y,lmd,delta):
    diff = delta + 1
    while diff > delta:
        b = np.sum(y-np.matmul(X,w))/len(X)
        curr_fn_val = fn_opt(X,w,y,b,lmd)
        diff = 0
        for k in range(len(X[0])):
            a = 2*np.dot(X[:,k],X[:,k])
            ek = np.transpose(np.eye(1,len(X[0]),k))
            btemp = y-(b+np.matmul(X,w)-w[k][0]*np.matmul(X,ek))
            c = 2*np.sum(np.matmul(X,ek)*btemp)
            if c < -1*lmd:
                diff = max(diff,np.absolute(w[k][0]-(c+lmd)/a))
                w[k][0] = (c+lmd)/a
            elif c > lmd:
                diff = max(diff,np.absolute(w[k][0]-(c-lmd)/a))
                w[k][0] = (c-lmd)/a
            else:
                diff = max(diff,np.absolute(w[k][0]))
                w[k][0] = 0
        new_fn_val = fn_opt(X,w,y,b,lmd)
        # a sanity check to ensure that the value of the objective function only decreases after every step
        if new_fn_val - curr_fn_val > 0:
            return
    
    return w,new_fn_val

# returns the L_2 and the L_1 difference between the predicted y (Xw) and the actual y
def fn_error(X,w,y):
    diff = np.matmul(X,w)-y
    return np.sum(diff*diff),np.sum(np.absolute(diff))

# the function that brings about cross validations
def lasso(X_train,y_train,X_validate,y_validate):
    yscale = y_train - np.reshape((np.sum(y_train)/len(y_train))*np.ones(len(y_train)),(len(y_train),1))
    
    # find the value of lambda that corresponds to the optimal value of w to be the all-zeros vector
    lmd = 0
    for k in range(len(X_train[0])):
        lmd = max(lmd,np.absolute(np.sum(yscale*np.reshape(X_train[:,k],(len(X_train),1)))))
    lmd = 2*lmd
    w = np.reshape(np.zeros(len(X_train[0])),(len(X_train[0]),1))
    
    # we maintain a list for values of lambda along with the corresponding w
    wlist = []
    lmdlist = []
    
    w,fn_val = lasso_fixlmd(X_train,w,y_train,lmd,2)
    
    curr_error = fn_error(X_validate,w,y_validate)
    new_error = curr_error - 50
    # we maintian a list for the validation and training error
    verror = [curr_error]
    trerror = [fn_error(X_train,w,y_train)]
    
    nzcount = np.count_nonzero(w)
    wlist.append(nzcount)
    lmdlist.append(lmd)
    lmd = lmd/1.5
    
    # we terminate when the validation error starts increasing
    # this gives an indication of the point where overfitting starts
    while new_error <= curr_error:
        curr_error = new_error
        
        w,fn_val = lasso_fixlmd(X,w,y,lmd,2)
        
        new_error = fn_error(X_validate,w,y_validate)
        verror.append(new_error)
        trerror.append(fn_error(X_train,w,y_train))
        
        nzcount = np.count_nonzero(w)
        wlist.append(nzcount)
        lmdlist.append(lmd)
        
        lmd = lmd/1.5
        print("Current Error is:", new_error)
    
    return wlist,lmdlist,w,verror,trerror

# we first import the data
X,y,featureNames = import_data()
y = np.sqrt(y)

# we make y a matrix
y=np.reshape(y,(len(y),1))

# train, validation and test split
X_train = X[:4000]
y_train = y[:4000]

X_validate = X[4000:5000]
y_validate = y[4000:5000]

X_test = X[5000:]
y_test = y[5000:]

# the following code just checks if the algorithms work when w is set to be the all zeros vector
#wtmp = np.reshape(np.zeros(len(X_train[0])),(len(X_train[0]),1))
#tmpw, tmpf = lasso_fixlmd(X_train,wtmp,y_train,10,0.01)
#print(fn_error(X_test,tmpw,y_test))
#print(tmp_error)

# the value of w chosen corresponds to the point where the validation error increases
wlist,lmdlist,w,verror,trerror = lasso(X_train,y_train,X_validate,y_validate)

# part (a)
plt.figure(1)
plt.plot(lmdlist,wlist,label="sparsity")
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Sparsity')
plt.show()

print("Test error =",test_error)

plt.figure(2)
plt.plot(lmdlist,verror, label = "validation error")
plt.plot(lmdlist,trerror, label = "train error")
plt.xscale('log')
plt.xlabel('Lambda')
plt.ylabel('Errors')
plt.show()

# part (b)

test_error = fn_error(X_test,w,y_test)
print('Test Error =',test_error)
train_error = fn_error(X_train,w,y_train)
print('Train Error =',train_error)
validation_error = fn_error(X_validate,w,y_validate)
print('Validations Error =',validation_error)

# part (c)

print('The top 10 features')
wlst = np.reshape(w,(1,len(w)))
top_ten = np.argsort(-np.absolute(wlst[0]))[:10]
for k in top_ten:
    print(featureNames[k])

(1050.7581461342538, 787.6090851281504)


In [ ]:
import numpy as np

a = np.ones(6)
a = np.reshape(a,(6,1))
print(a*a)
print(np.sum(a*a))
print(a+1)
print(a[1])
b = np.reshape(a,(3,2))
print(b)
print(np.asmatrix(b[:,1]))
print(np.eye(1,5,2))
print(np.transpose(np.eye(1,5,2)))
print(a)
atemp = np.reshape(a,(1,6))
print(atemp[0])

In [55]:
print(w.shape)
wlst = np.reshape(w,(1,len(w)))
#top_ten = np.argpartition(w,-10)[-10:0]
top_ten = np.argsort(-np.absolute(wlst[0]))[:10]
print(top_ten)
print(w[top_ten])
#print(featureNames[top_ten])
print(type(featureNames))
print(top_ten.astype(int))
print(featureNames[0])
for k in top_ten:
    print(featureNames[k])
print(featureNames[top_ten.astype(int)])

(1000, 1)
[190   0  81  13 176 127 353 266 646 605]
[[11.92244038]
 [ 5.30307668]
 [ 4.26125746]
 [ 3.9035341 ]
 [-3.84569876]
 [ 3.30557423]
 [ 2.98239195]
 [ 1.81497438]
 [ 1.51967414]
 [ 1.38103878]]
<class 'list'>
[190   0  81  13 176 127 353 266 646 605]
sq(UserFunnyVotes*BusinessNumStars)
sqrt(ReviewNumCharacters*UserFunnyVotes)
sq(UserFunnyVotes*BusinessNumStars)
sqrt(UserFunnyVotes*InPhoenix)
log(ReviewNumLineBreaks*BusinessNumReviews)
ReviewNumCharacters*BusinessLongitude
log(ReviewNumLineBreaks*BusinessNumStars)
sqrt(UserCoolVotes*BusinessNumStars)
sqrt(UserFunnyVotes*BusinessNumStars)
log(UserUsefulVotes)
log(ReviewNumCharacters*UserUsefulVotes)


TypeError: only integer scalar arrays can be converted to a scalar index

In [9]:
print(w)
print(fn_error(X_test,w,y_test))
print(fn_error(X_validate,w,y_validate))
print(fn_error(X_train,w,y_train))

[[-4.97092871e+00]
 [ 5.48837536e-01]
 [-2.32765524e+00]
 [ 5.64337109e-02]
 [ 2.84004681e-01]
 [ 1.27750817e+00]
 [-8.83649600e-02]
 [-1.57706309e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.63335015e-02]
 [-2.52453412e-01]
 [ 2.35521812e+00]
 [-2.75541156e-01]
 [ 2.21781298e-01]
 [-3.82338680e-01]
 [-6.93044812e-02]
 [ 1.09446438e-01]
 [-2.85256810e-02]
 [-1.19995425e-01]
 [-8.08027778e-17]
 [ 1.36184136e-01]
 [-1.27091579e+00]
 [-1.23865203e-01]
 [-3.95389394e-02]
 [ 7.81757551e-02]
 [-4.76803840e-02]
 [ 2.96768424e-01]
 [ 2.10551407e-02]
 [-6.14229153e-03]
 [-1.14422933e+00]
 [ 5.74554333e-01]
 [ 4.06148761e-01]
 [ 3.82032749e-02]
 [-1.23044782e-01]
 [ 5.43036631e-02]
 [ 1.28699573e+00]
 [-7.29111427e-02]
 [-1.86201800e-02]
 [-2.11990243e+00]
 [-7.60355479e-02]
 [ 1.29874045e-01]
 [ 1.46653684e-01]
 [-1.49043939e-01]
 [-5.63498558e-01]
 [-1.10045318e+00]
 [ 3.34835197e-01]
 [-3.19131578e-01]
 [-1.13966239e+00]
 [ 1.39896061e-01]
 [ 2.49856663e+00]
 [-1.0300288